## 矩阵分解or 矩阵补充算法（过时&不适合做召回）

原理
1) 计算物品emb 和用户emb
2) 使用dot计算 user_emb, item_emb见的内积
3) 内积结果用来拟合user给item的打分值

缺点：
1) 仅⽤ID embedding，没利⽤物品、⽤户属性
2) 负样本：曝光之后，没有点击、交互。（错误的做法）
3) 做训练的⽅法不好(回归不如分类，内积不如余弦相似度)
4) 对新物品不友好( 新物品没有展现数据，rate数据，更没有训练好的emb)


In [1]:
%cd /playground/sgd_deep_learning/sgd_rec_sys/
import sys 
sys.path.append('./python')

/playground/sgd_deep_learning/sgd_rec_sys


In [2]:
import numpy as np
import random
import torch
from sgd_rec_sys.retrieval import MF, RateInfo

## 准备数据

In [3]:
# TODO: 改写成DataSet 和 DataLoader
def load_data():
    data = [(1, 2, 4.5),
            (1, 3, 2.0),
            (2, 1, 4.0), 
            (2, 3, 3.5),
            (3, 2, 5.0),
            (3, 4, 2.0),
            (4, 2, 3.5), 
            (4, 3, 4.0), 
            (4, 4, 1.0)]
    uids = [x[0] - 1 for x in data]
    iids = [x[1] - 1 for x in data]
    scores = [x[2] for x in data]

    return torch.LongTensor(uids), torch.LongTensor(iids),  torch.Tensor(scores)

user_ids, item_ids, scores = load_data() 
print(user_ids, item_ids, scores)

tensor([0, 0, 1, 1, 2, 2, 3, 3, 3]) tensor([1, 2, 0, 2, 1, 3, 1, 2, 3]) tensor([4.5000, 2.0000, 4.0000, 3.5000, 5.0000, 2.0000, 3.5000, 4.0000, 1.0000])


In [4]:
def train(model, train_x, loss_fn, optimizer):
    user_ids, item_ids, scores = train_x
    y = model.forward(user_ids, item_ids)
    out = loss_fn(y, scores)

    # Backpropagation
    optimizer.zero_grad()
    out.backward()
    optimizer.step()
    return out

In [5]:
mf = MF(userN=4, itemN=4, dim=32)
print(mf)

training_data = load_data()
loss_fn = torch.nn.MSELoss()
# optimizer = torch.optim.SGD(mf.parameters(), lr=1e-2)


best_states = None
best_loss = None

for epoch in range(30):
    optimizer = torch.optim.Adam(mf.parameters(), lr=1e-2, weight_decay=0.1)
    loss = train(model=mf, train_x=training_data, loss_fn=loss_fn, optimizer=optimizer)
    
    if best_loss is not None and loss > best_loss:
        continue
    else:
        best_loss = loss
        best_states = mf.state_dict()
    
    print(epoch, 'loss:  {:.4f}'.format(loss.item()))

# load best parameters
mf.load_state_dict(best_states)

MF(
  (user_emb): Embedding(4, 32)
  (item_emb): Embedding(4, 32)
)


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0 loss:  34.4774
1 loss:  30.5983
2 loss:  26.9791
3 loss:  23.6103
4 loss:  20.4892
5 loss:  17.6141
6 loss:  14.9820
7 loss:  12.5897
8 loss:  10.4347
9 loss:  8.5119
10 loss:  6.8206
11 loss:  5.3488
12 loss:  4.0808
13 loss:  3.0087
14 loss:  2.1172
15 loss:  1.4100
16 loss:  0.8663
17 loss:  0.4704
18 loss:  0.2075
19 loss:  0.0634
20 loss:  0.0100
21 loss:  0.0052


<All keys matched successfully>

## 验证训练集效果


In [6]:
# 查看随机样本预测情况
k = random.randint(0, len(training_data[0])-1)
user_id, item_id, rate_score = [training_data[i][k] for i in range(3)]

pred_y = mf.forward(user_id.unsqueeze(-1), item_id.unsqueeze(-1))
print("user[{}], item[{}], pred_rate[{}], True_rate[{:.4f}]".format(user_id, item_id, rate_score, pred_y.item()))

user[2], item[3], pred_rate[2.0], True_rate[2.0666]


## emb线上使用流程
* user emb 使用userid 即可查找
* item emb 需要使用Milvus、Faiss、HnswLib等向量数据库，方便快速找到与query(user emb)内积最大的topk物品